In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-09-30 10:27:27--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-Collisions.csv.6’

Data-Collisions.csv 100%[===================>]  70.49M  24.6MB/s    in 2.9s    

2020-09-30 10:27:30 (24.6 MB/s) - ‘Data-Collisions.csv.6’ saved [73917638/73917638]



In [3]:
df=pd.read_csv("Data-Collisions.csv")

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [5]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [6]:
# 1. Data Preparation
# Drop the columns we do not need in our module to decrease the data analysis cost
dasta = df.drop(columns={'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
                        'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
                        'X', 'Y','EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC',
                        'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT',
                        'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
                        'INATTENTIONIND', 'UNDERINFL',
                        'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
                        'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'})
# convert the independent variables into categories rather than objective 
dasta["WEATHER"] = dasta["WEATHER"].astype('category')
dasta["ROADCOND"] = dasta["ROADCOND"].astype('category')
dasta["LIGHTCOND"] = dasta["LIGHTCOND"].astype('category')
#Coding the independent variables
dasta["WEATHER_CAT"] = dasta["WEATHER"].cat.codes
dasta["ROADCOND_CAT"] = dasta["ROADCOND"].cat.codes
dasta["LIGHTCOND_CAT"] = dasta["LIGHTCOND"].cat.codes
dasta.dtypes

SEVERITYCODE        int64
WEATHER          category
ROADCOND         category
LIGHTCOND        category
WEATHER_CAT          int8
ROADCOND_CAT         int8
LIGHTCOND_CAT        int8
dtype: object

In [7]:
dasta.SEVERITYCODE.value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [8]:
#Balance the data set  
from sklearn.utils import resample
data_more = dasta[dasta.SEVERITYCODE ==1]
data_less = dasta[dasta.SEVERITYCODE ==2]
data_more_equal = resample(data_more,
                                      replace=False,
                                      n_samples=58188,
                                      random_state=99)
data_bal = pd.concat([data_more_equal,data_less])
data_bal.SEVERITYCODE.value_counts()

2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

In [9]:
from sklearn import preprocessing
# determine x = the independent varaibles and y as the dependent variable
x = np.asarray(data_bal[['WEATHER_CAT','ROADCOND_CAT','LIGHTCOND_CAT']])
y = np.asarray(data_bal['SEVERITYCODE'])
print ('The value of x before preporcessing ',x[0:5])
print ('The value of x before preporcessing ',y[0:5])

x=preprocessing.StandardScaler().fit(x).transform(x)
print ('The value of x after preporcessing ',x[0:5])

The value of x before preporcessing  [[ 6  8  5]
 [ 1  0  5]
 [ 6  8  5]
 [10  7  8]
 [ 6  8  5]]
The value of x before preporcessing  [1 1 1 1 1]
The value of x after preporcessing  [[ 1.15109535  1.53056569  0.42621151]
 [-0.67433413 -0.67001234  0.42621151]
 [ 1.15109535  1.53056569  0.42621151]
 [ 2.61143893  1.25549344  2.07234651]
 [ 1.15109535  1.53056569  0.42621151]]


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [10]:
# Split the data set into train (80%) and test (20%)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

In [11]:
# Logistic regression 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

LR = LogisticRegression(C=0.01, solver='liblinear').fit(x_train,y_train)
LR
yhat_LR = LR.predict(x_test)
print('Logistic regression = ',yhat_LR) 

yhat_prob_LR = LR.predict_proba(x_test)
print('Logistic regression probability = ',yhat_prob_LR) 

Logistic regression =  [2 1 2 ... 2 2 2]
Logistic regression probability =  [[0.47262299 0.52737701]
 [0.68377027 0.31622973]
 [0.4655792  0.5344208 ]
 ...
 [0.47262299 0.52737701]
 [0.47262299 0.52737701]
 [0.47262299 0.52737701]]


In [15]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
k = 15
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(x_train,y_train)
neigh
yhat_KNN = neigh.predict(x_test)
yhat_KNN[0:5]

array([2, 1, 1, 1, 1])

In [13]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
DecTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
DecTree
DecTree.fit(x_train,y_train)
pred_DT = DecTree.predict(x_test)
print (pred_DT [0:5])
print (y_test [0:5])

[2 1 2 2 2]
[2 1 2 1 2]


In [16]:
# Evaluate the three models to see which one is more accurate 
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

#1.Evaluate LR model
LRjaccard = jaccard_similarity_score(y_test, yhat_LR)
print ('LR jaccard_similarity_score = ',LRjaccard)
LRlog_loss = log_loss(y_test, yhat_prob_LR)
print ('LR log_loss = ',LRlog_loss)
LR_f1_score= f1_score(y_test, yhat_LR, average='macro')
print ('LR F1 score = ',LR_f1_score)
print ('***********************')
#2. Evaluate KNN model 
KNNjaccard = jaccard_similarity_score(y_test, yhat_KNN)
print ('KNN jaccard_similarity_score = ',KNNjaccard)
KNN_f1_score= f1_score(y_test, yhat_KNN, average='macro')
print ('KNN F1 score = ',KNN_f1_score)
print ('========================')
#3.Decision Tree Model 
DTjaccard = jaccard_similarity_score(y_test, pred_DT)
print ('Decision Tree jaccard_similarity_score = ',DTjaccard)
DT_f1_score= f1_score(y_test, pred_DT, average='macro')
print ('Decision Tree F1 score = ',DT_f1_score)

LR jaccard_similarity_score =  0.5295583433579653
LR log_loss =  0.6843902239068189
LR F1 score =  0.5142657138878495
***********************
KNN jaccard_similarity_score =  0.5566248496305207
KNN F1 score =  0.5449387158254655
Decision Tree jaccard_similarity_score =  0.5648307269290256
Decision Tree F1 score =  0.48361182945584147
